In [1]:
# coding:utf8
import pandas as pd
import time
import numpy as np
import urllib, json

In [2]:
import seaborn as sns

In [3]:
class PropertySamp(object):
    def getData(self):
        url = 'https://htem-api.nrel.gov/api/sample/'+str(self.id)+'/mvl_opt'
        response = urllib.request.urlopen(url)
        data = json.loads(response.read())
        wl0=data['library-spectra-measurement']['data'][0]['wavelength']['value']
        rf=data['library-spectra-measurement']['data'][0]['reflectance']['value']
        wl1=data['library-spectra-measurement']['data'][1]['wavelength']['value']
        tr=data['library-spectra-measurement']['data'][1]['transmittance']['value']
        df0=pd.DataFrame({'reflectance':rf},index=wl0)
        self.rf=df0.stack().unstack(0)
        df1=pd.DataFrame({'transmittance':tr},index=wl1)
        self.tr=df1.stack().unstack(0)
        url = 'https://htem-api.nrel.gov/api/sample/'+str(self.id)+'/mvl_ele'
        #There is the potential to replace this with mvl_optical or mvl_xrd, 
            #but these seem to be broken at the moment...
        response = urllib.request.urlopen(url)
        data = json.loads(response.read())
        rs=data['library-conductivity-resistivity-measurement']['data'][0]['resistivity']['value']
        cd=data['library-conductivity-resistivity-measurement']['data'][0]['conductivity']['value']
        sr=data['library-conductivity-resistivity-measurement']['data'][0]['sheet-resistance']['value']
        self.ele=pd.DataFrame({'resistivity':[rs],'conductivity':[cd],'sheet-resistance':[sr]})
        
    def __init__(self,id):
        self.id=id
        self.getData()

In [5]:
class PropertyLib(object):
    def __init__(self,id):
        self.id = id
        self.sample=[]
        self.getData()
    def getData(self):
        url = 'https://htem-api.nrel.gov/api/sample_library/'+str(self.id)
        response = urllib.request.urlopen(url)
        data = json.loads(response.read())
        positions = data['sample_ids']
        for p in positions:
            self.sample.append(PropertySamp(p))
        index=[s.id for s in self.sample]
        self.rf=pd.concat([s.rf for s in self.sample],axis=0)
        self.rf.index=index
        self.tr=pd.concat([s.tr for s in self.sample],axis=0)
        self.tr.index=index
        self.ele=pd.concat([s.ele for s in self.sample],axis=0)
        self.ele.index=index
       
        

In [6]:
class PropertyELESamp(object):
    def getData(self):
        url = 'https://htem-api.nrel.gov/api/sample/'+str(self.id)+'/mvl_ele'
        response = urllib.request.urlopen(url)
        data = json.loads(response.read())
        rs=data['library-conductivity-resistivity-measurement']['data'][0]['resistivity']['value']
        cd=data['library-conductivity-resistivity-measurement']['data'][0]['conductivity']['value']
        sr=data['library-conductivity-resistivity-measurement']['data'][0]['sheet-resistance']['value']
        x=data['library-conductivity-resistivity-measurement']['point']['coordinates']['x']['value']
        y=data['library-conductivity-resistivity-measurement']['point']['coordinates']['y']['value']
        self.ele=pd.DataFrame({'resistivity':[rs],'conductivity':[cd],'sheet-resistance':[sr],'x':[x],'y':[y]})
        
    def __init__(self,id):
        self.id=id
        self.getData()

In [7]:
class PropertyELELib(object):
    def __init__(self,id):
        self.id = id
        self.sample=[]
        self.getData()
    def getData(self):
        url = 'https://htem-api.nrel.gov/api/sample_library/'+str(self.id)
        response = urllib.request.urlopen(url)
        data = json.loads(response.read())
        positions = data['sample_ids']
        for p in positions:
            self.sample.append(PropertyELESamp(p))
        index=[s.id for s in self.sample]
        self.ele=pd.concat([s.ele for s in self.sample],axis=0)
        self.ele.index=index

In [18]:
elelab=PropertyELELib(6651)

In [19]:
ele=elelab.ele

In [20]:
ele['z']=ele['sheet-resistance']/ele.resistivity

In [21]:
ele=ele[['x','y','z','resistivity']]

In [22]:
ele['x']=ele['x'].map(float)

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
ele['y']=ele['y'].map(float)

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
ele['id']=ele.index

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
ele.to_json('lib6651.json',orient="values")

In [8]:
ss=PropertyLib(6651)

下面是因为原数据中存在着波长不对应的脏数据，我们只取相同波长取得。

In [17]:
for s in ss.tr.index:
    singlesample=PropertySamp(s)
    tmp=singlesample.tr.stack().unstack(0)
    tmp['in']=[k-int(k) for k in tmp.index ]
    cleantr=tmp[tmp['in']==0.0]
    cleanrf=singlesample.rf.stack().unstack(0)
    cleandata=singlesample.tr.stack().unstack(0)
    cleandata=cleanrf.merge(cleandata,left_index=True,right_index=True)
    cleandata['absorbility']=1-cleandata.reflectance-cleandata.transmittance
    cleandata['wavelength']=cleandata.index
    cleandata.stack().unstack(0).to_json(str(s)+'.json',orient='values')